In [1]:
import numpy as np
import pandas as pd
import sklearn
import mglearn
from sklearnex  import  patch_sklearn 
patch_sklearn ()


dataset = pd.read_csv('dataset_limpio.csv')
dataset = dataset.drop(['Unnamed: 0'], axis=1)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from sklearnex  import  patch_sklearn 
patch_sklearn ()
from time import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import warnings

def plot_figure(data, labels):
    

    
    plt.figure(figsize=(8, 5))
    plt.scatter(data[:,0], data[:,1], c=labels, linewidths= 0.01, edgecolors='k')
    
    plt.savefig("spectral.pdf")


def bench_marks(algoritm, name, data):
    

    t0 = time()
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="Graph is not fully connected, spectral embedding"
            + " may not work as expected.",
            category=UserWarning,
        )
        estimator = algoritm.fit(data)
    labels= estimator.labels_
    fit_time = time() - t0    

    results = [name, fit_time]

    results += [silhouette_score(data, labels, metric = 'euclidean')]
    results += [davies_bouldin_score(data,labels)]
    results += [calinski_harabasz_score(data, labels)]
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    results += [n_clusters]
    
    formatter_result = (
        "{:5s}\t\t{:.3f}s\t\t{:.3f}\t{:.3f}\t\t{:.2f}\t\t{:.0f}"
    )
    print(formatter_result.format(*results))
    


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from sklearn.cluster import SpectralClustering
from sklearnex  import  patch_sklearn 
patch_sklearn ()

random_state = np.random.seed(42)
n_clusters = [2, 3]
print(82 * "_")
print("assign_labels\t\t\ttime\t\tsilhouette\tdavies_bouldin\tcalinski_harabasz\tn_clusters\n")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset)
pca = PCA(n_components = 0.95)
X_pca = pca.fit_transform(X_scaled)

for i in n_clusters:
    spectralDiscretize = SpectralClustering(n_clusters = i, assign_labels = 'discretize', affinity = 'nearest_neighbors', random_state = random_state, n_jobs = -1, eigen_solver = 'amg', n_neighbors = 2)
    bench_marks(spectralDiscretize, name="discretize", data = X_pca)

    spectralcluster_qr = SpectralClustering(n_clusters = i, assign_labels = 'cluster_qr', affinity = 'nearest_neighbors', random_state = random_state, n_jobs = -1, eigen_solver = 'amg', n_neighbors = 2)
    bench_marks(spectralcluster_qr, name="cluster_qr", data = X_pca)

    spectralkmeans = SpectralClustering(n_clusters = i, assign_labels = 'kmeans', affinity = 'nearest_neighbors', random_state = random_state, n_jobs=-1, eigen_solver = 'amg', n_neighbors = 2)
    bench_marks(spectralkmeans, name="kmeans", data = X_pca)

estimator = spectralkmeans.fit(X_pca)
labels= estimator.labels_
plot_figure(X_pca, labels)


print(82 * "_")

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


__________________________________________________________________________________
assign_labels			time		silhouette	davies_bouldin	calinski_harabasz	n_clusters

